# **Scale Factor Calculation: GNSS Coordinates**

This document explains the Python code within the GNSS_ScaleFactor.ipynb notebook. The primary goal of the script is to convert a set of UTM (Universal Transverse Mercator) coordinates into geographic coordinates (latitude and longitude) and then calculate the corresponding projection scale factors for that location.

#### ***Setup and Initialization***

Installation: The **pyproj** library is installed. This library is essential for performing geodetic calculations and coordinate transformations.

Import the required modules. ***Proj*** is the core class from the **pyproj** library used to define and work with cartographic projections:

In [6]:
!pip install pyproj

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.3 MB ? eta -:--:--
   ----- ---------------------------------- 0.8/6.3 MB 1.6 MB/s eta 0:00:04
   -------- ------------------------------- 1.3/6.3 MB 1.6 MB/s eta 0:00:04
   ---------- ----------------------------- 1.6/6.3 MB 1.5 MB/s eta 0:00:04
   ----------- ---------------------------- 1.8/6.3 MB 1.5 MB/s eta 0:00:03
   --------------- ------------------------ 2.4/6.3 MB 1.6 MB/s eta 0:00:03
   ------------------ --------------------- 2.9/6.3 MB 1.7 MB/s eta 0:00:02
   -------------------- ------------------- 3.1/6.3 MB 1.8 MB/s eta 0:00:02
   ------------------------- -------------- 3.9/6.3 MB 1.9 MB/s eta 0:00:02
   ---------------------------- ----------- 4.5/6.3 MB 2.0 MB/s eta 0:00:01
   ------------------------------ --------- 4.7/6.3 MB 1.9 MB/s eta 0:00:01
   ------------------------------

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj

#### ***Coordinate Conversion: UTM to Geographic***
This section takes a known point in UTM coordinates and converts it to its latitude and longitude equivalent.

***Input Coordinates:*** The script starts with a defined point in UTM Zone 12, using the WGS84 ellipsoid as the reference frame.

***Transformation:*** A Proj object is configured for UTM Zone 12. The inverse=True argument in the next step tells the function to convert from UTM to geographic coordinates. The result is the latitude and longitude for the input easting and northing.

In [30]:
# UTM coordinates:
east_utm = 540838.837
north_utm = 3369972.004
elev = 1277.410 #Orthometric

# Mexico: Calculation between orthometric and ellipsoidal heigth check: https://www.inegi.org.mx/app/geo2/alturasgeoidales.

# UTM projection definition (Meridian: 111W meridian):
utm = Proj(proj='utm', zone=12, ellips ='WGS84')

# Lat/Long conversion:
longitude, latitude = utm(east_utm, north_utm, inverse=True)

print(f"Latitude: {latitude:.8f}°")
print(f"Longitude: {longitude:.8f}°")

Latitude: 30.46122976°
Longitude: -110.57459100°


#### ***Scale Factor Calculation***
The core logic of the notebook is encapsulated in the utm_scalefactor function. This function calculates the factors needed to convert a true ground distance to its representation on the UTM map grid.

Point Scale Factor (k): This factor corrects for the distortion inherent in the UTM projection. Because a map is a flat representation of the curved Earth, distances are slightly distorted. The scale factor k is 1.0 on the standard meridians and changes as you move away from them. In the code, this is referred to as map_factor.

Elevation Factor (SLF): A distance measured on the ground at a certain elevation (H) is longer than its equivalent distance on the reference ellipsoid (which approximates sea level). The Sea Level Factor reduces the ground distance to the ellipsoid. It is calculated with the formula SLF = R / (R + H), where R is the mean radius of the Earth.

Combined Factor (CF): This is the product of the Point Scale Factor and the Elevation Factor (CF = k * SLF). It is the final multiplier used to convert a true horizontal ground distance to a "grid distance" on the map.

The function is defined as follows:

In [31]:
def utm_scalefactor(lat, long, elev):
    # UTM Scale Factor
    factors = utm.get_factors(long, lat)
    k = factors.meridional_scale
    slf = 6378000/(6378000 + elev) #Sea Level Factor: SLF=R/(R+H)
    cf = k*slf
    return cf, k, slf

#### ***Results***
Finally, the script calls the function with the calculated latitude/longitude and a specific ground elevation (1247.300 m) to get the correction factors for that exact point.

In [32]:
cf, k, slf = utm_scalefactor(30.46122976, -110.5745910, 1247.300) # Ellipsoidal height calculated.
print(f"Map Factor: {k:.6f}, SLF: {slf:.6f}")
print(f"Map Distance x 100 meters: {k*100:.6f}")

Map Factor: 0.999621, SLF: 0.999804
Map Distance x 100 meters: 99.962057


***Output:*** The script prints the individual map_factor and slf. The final line demonstrates that a 100-meter distance on the ellipsoid would be represented as approximately 99.962 meters on the UTM grid at this specific location (100 * 0.999621). To get the full correction from a ground distance, one would multiply by the combined factor.